In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

# Day 4: CUDA Events - Stopwatches & Checkpoints

---

## 🎯 The Hook: The Relay Race Baton Pass

**Think about** a relay race. Each runner needs to:
1. Know when the previous runner arrives (synchronization)
2. Measure their own lap time (timing)
3. Signal the next runner to start (coordination)

```
Without Events (everyone runs blindly):
┌───────────────────────────────────────────────────────────────┐
│  Runner 1: [Run]─────────────────────────────────────────>    │
│  Runner 2:         [Run]─────────────────> (started too early!)│
│  Runner 3:                    [Run]───> (collision!)          │
│                                                               │
│  Result: CHAOS! 🔥                                            │
└───────────────────────────────────────────────────────────────┘

With Events (coordinated handoffs):
┌───────────────────────────────────────────────────────────────┐
│  Runner 1: [Run]──🏁 event1                                   │
│  Runner 2:         wait(event1)──[Run]──🏁 event2             │
│  Runner 3:                        wait(event2)──[Run]──🏁     │
│                                                               │
│  Result: Perfect relay! Each runner timed precisely! ⏱️       │
└───────────────────────────────────────────────────────────────┘
```

**CUDA Events are your batons** - they synchronize work AND measure time with GPU precision!

---

## 📚 Learning Objectives

By the end of this lesson, you will be able to:
1. **Create and record** events to mark points in streams
2. **Measure elapsed time** between events with microsecond accuracy
3. **Synchronize streams** using `cudaStreamWaitEvent()`
4. **Query event status** without blocking the CPU
5. **Design DAG workflows** where some streams depend on others

---

## 🃏 Concept Card: Stopwatches & Checkpoints

```
╔══════════════════════════════════════════════════════════════════╗
║  ⏱️  CUDA EVENTS = STOPWATCHES + CHECKPOINTS                     ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  DUAL PURPOSE:                                                   ║
║                                                                  ║
║  1️⃣  STOPWATCH: Measure GPU kernel time                          ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  🏁 start                        🏁 stop                   │  ║
║  │    ↓                               ↓                       │  ║
║  │  [Record] → [Kernel Runs] → [Record] → [Elapsed Time!]     │  ║
║  │                                                            │  ║
║  │  cudaEventElapsedTime(&ms, start, stop)  → "42.5 ms"       │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  2️⃣  CHECKPOINT: Coordinate between streams                      ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  Stream A: [Produce Data]─🏁 event ─────────────────────   │  ║
║  │                            │                               │  ║
║  │  Stream B: ────────────────wait(event)─[Consume Data]      │  ║
║  │                                                            │  ║
║  │  "Stream B waits for Stream A to reach the checkpoint"     │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  💡 KEY INSIGHT: Events are recorded in streams but can be       ║
║     waited on by ANY stream - enabling complex dependencies!     ║
║                                                                  ║
║  TIMING ACCURACY:                                                ║
║  • CPU timers:   ~1ms resolution, affected by OS scheduling      ║
║  • CUDA events:  ~0.5μs resolution, GPU clock precision!         ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## Part 1: CUDA Events Basics

### What are CUDA Events?

```
EVENT = A marker in a stream that can be:
        • Recorded (placed at a point in stream)
        • Queried (check if reached)
        • Waited on (block until reached)
        • Used for timing

Stream Timeline:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
[H2D Copy] → [event1] → [Kernel] → [event2] → [D2H Copy]
              ↑                      ↑
        record start           record stop
              └─────────────────────┘
                 elapsed time!
```

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates event creation, recording, synchronization, and timing.

In [ ]:
%%writefile event_basics.cu
// event_basics.cu - Event creation, recording, and synchronization
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void computeKernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 100; i++) {
            data[tid] = sqrtf(data[tid] + 1.0f);
        }
    }
}

int main() {
    const int N = 1 << 22;
    
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    // ============================================
    // Event Creation
    // ============================================
    cudaEvent_t start, stop;
    
    // Default events (for timing and sync)
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Event with flags:
    // cudaEventDefault      - Default behavior
    // cudaEventBlockingSync - CPU blocks instead of spinning
    // cudaEventDisableTiming- Faster, but can't time
    // cudaEventInterprocess - For IPC
    
    cudaEvent_t fastEvent;
    cudaEventCreateWithFlags(&fastEvent, cudaEventDisableTiming);
    
    // ============================================
    // Event Recording (place marker in stream)
    // ============================================
    cudaEventRecord(start);  // Default stream
    // OR: cudaEventRecord(start, stream);  // Specific stream
    
    computeKernel<<<256, 256>>>(d_data, N);
    computeKernel<<<256, 256>>>(d_data, N);
    computeKernel<<<256, 256>>>(d_data, N);
    
    cudaEventRecord(stop);
    
    // ============================================
    // Event Synchronization
    // ============================================
    
    // Method 1: Block until event completes
    cudaEventSynchronize(stop);
    
    // Method 2: Query (non-blocking)
    cudaError_t status = cudaEventQuery(stop);
    if (status == cudaSuccess) {
        printf("Event completed!\n");
    } else if (status == cudaErrorNotReady) {
        printf("Event not yet reached\n");
    }
    
    // ============================================
    // Event Timing
    // ============================================
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Kernel time: %.3f ms\n", milliseconds);
    
    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(fastEvent);
    cudaFree(d_data);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o event_basics event_basics.cu
!./event_basics

---

## Part 2: Accurate GPU Timing

### Why Events for Timing?

```
CPU Timing (WRONG for GPU):
━━━━━━━━━━━━━━━━━━━━━━━━━━━
auto start = clock();
kernel<<<>>>(...);        // Returns immediately!
auto end = clock();       // Measures launch overhead only
// WRONG: Measures ~0.001ms instead of actual kernel time

GPU Event Timing (CORRECT):
━━━━━━━━━━━━━━━━━━━━━━━━━━
cudaEventRecord(start);
kernel<<<>>>(...);        // Runs asynchronously
cudaEventRecord(stop);
cudaEventSynchronize(stop);
cudaEventElapsedTime(&ms, start, stop);
// CORRECT: Measures actual GPU execution time
```

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates proper GPU timing techniques including warmup, single kernel timing, averaging over multiple iterations, and timing individual sections.

In [ ]:
%%writefile accurate_timing.cu
// accurate_timing.cu - Proper GPU timing
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void kernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 1000; i++) {
            data[tid] = sinf(data[tid]);
        }
    }
}

int main() {
    const int N = 1 << 20;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // ============================================
    // Warmup (IMPORTANT!)
    // ============================================
    // First kernel launch has overhead (JIT, caching, etc.)
    kernel<<<256, 256>>>(d_data, N);
    cudaDeviceSynchronize();
    
    // ============================================
    // Time Single Kernel
    // ============================================
    cudaEventRecord(start);
    kernel<<<256, 256>>>(d_data, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float singleTime;
    cudaEventElapsedTime(&singleTime, start, stop);
    printf("Single kernel: %.3f ms\n", singleTime);
    
    // ============================================
    // Time Multiple Iterations (more accurate)
    // ============================================
    const int ITERATIONS = 100;
    
    cudaEventRecord(start);
    for (int i = 0; i < ITERATIONS; i++) {
        kernel<<<256, 256>>>(d_data, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float totalTime;
    cudaEventElapsedTime(&totalTime, start, stop);
    printf("Average over %d runs: %.3f ms\n", 
           ITERATIONS, totalTime / ITERATIONS);
    
    // ============================================
    // Time Individual Sections
    // ============================================
    cudaEvent_t e1, e2, e3;
    cudaEventCreate(&e1);
    cudaEventCreate(&e2);
    cudaEventCreate(&e3);
    
    float *h_data;
    cudaMallocHost(&h_data, N * sizeof(float));
    
    cudaEventRecord(e1);
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaEventRecord(e2);
    kernel<<<256, 256>>>(d_data, N);
    cudaEventRecord(e3);
    cudaEventSynchronize(e3);
    
    float h2dTime, kernelTime;
    cudaEventElapsedTime(&h2dTime, e1, e2);
    cudaEventElapsedTime(&kernelTime, e2, e3);
    
    printf("H2D transfer: %.3f ms\n", h2dTime);
    printf("Kernel exec:  %.3f ms\n", kernelTime);
    
    // Cleanup
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaEventDestroy(e1);
    cudaEventDestroy(e2);
    cudaEventDestroy(e3);
    cudaFree(d_data);
    cudaFreeHost(h_data);
    
    return 0;
}

---

## Part 3: Inter-Stream Synchronization

### cudaStreamWaitEvent

```
Without Event Sync:            With Event Sync:
━━━━━━━━━━━━━━━━━━━━           ━━━━━━━━━━━━━━━━━━━
Stream A: [Produce Data]       Stream A: [Produce]──┐
Stream B: [Use Data] ← RACE!   Stream B:            └─→[Wait][Use]
                                                    ↑
                                              cudaStreamWaitEvent
```

### 🔷 CUDA C++ Implementation (Primary)

The following example demonstrates:
- Producer-consumer pattern with event synchronization
- Complex dependency graphs (DAG) with multiple streams
- Proper cleanup of events and streams

In [ ]:
%%writefile inter_stream_sync.cu
// inter_stream_sync.cu - Synchronizing between streams
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void produce(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = (float)tid * 2.0f;  // Produce data
    }
}

__global__ void consume(const float* input, float* output, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        output[tid] = input[tid] + 1.0f;  // Use produced data
    }
}

int main() {
    const int N = 1 << 20;
    
    float *d_produced, *d_consumed;
    cudaMalloc(&d_produced, N * sizeof(float));
    cudaMalloc(&d_consumed, N * sizeof(float));
    
    cudaStream_t streamA, streamB;
    cudaStreamCreate(&streamA);
    cudaStreamCreate(&streamB);
    
    // Event for synchronization
    cudaEvent_t dataReady;
    cudaEventCreate(&dataReady);
    
    // ============================================
    // Stream A: Produce data
    // ============================================
    produce<<<256, 256, 0, streamA>>>(d_produced, N);
    
    // Record event when production is done
    cudaEventRecord(dataReady, streamA);
    
    // ============================================
    // Stream B: Wait for data, then consume
    // ============================================
    // This makes streamB wait until streamA reaches dataReady
    cudaStreamWaitEvent(streamB, dataReady);
    
    consume<<<256, 256, 0, streamB>>>(d_produced, d_consumed, N);
    
    // Both streams can do other independent work before/after
    
    cudaDeviceSynchronize();
    printf("Producer-consumer with event sync complete!\n");
    
    // ============================================
    // Complex Dependency Graph
    // ============================================
    //       A1
    //      /  \
    //     B1  B2
    //      \  /
    //       C1
    
    cudaStream_t sA, sB1, sB2, sC;
    cudaStreamCreate(&sA);
    cudaStreamCreate(&sB1);
    cudaStreamCreate(&sB2);
    cudaStreamCreate(&sC);
    
    cudaEvent_t afterA, afterB1, afterB2;
    cudaEventCreate(&afterA);
    cudaEventCreate(&afterB1);
    cudaEventCreate(&afterB2);
    
    // A1: Initial work
    produce<<<256, 256, 0, sA>>>(d_produced, N);
    cudaEventRecord(afterA, sA);
    
    // B1 and B2: Parallel work after A
    cudaStreamWaitEvent(sB1, afterA);
    cudaStreamWaitEvent(sB2, afterA);
    consume<<<256, 256, 0, sB1>>>(d_produced, d_consumed, N/2);
    consume<<<256, 256, 0, sB2>>>(d_produced + N/2, d_consumed + N/2, N/2);
    cudaEventRecord(afterB1, sB1);
    cudaEventRecord(afterB2, sB2);
    
    // C1: After both B1 and B2 complete
    cudaStreamWaitEvent(sC, afterB1);
    cudaStreamWaitEvent(sC, afterB2);
    produce<<<256, 256, 0, sC>>>(d_consumed, N);  // Final processing
    
    cudaDeviceSynchronize();
    printf("Complex dependency graph complete!\n");
    
    // Cleanup
    cudaEventDestroy(dataReady);
    cudaEventDestroy(afterA);
    cudaEventDestroy(afterB1);
    cudaEventDestroy(afterB2);
    cudaStreamDestroy(streamA);
    cudaStreamDestroy(streamB);
    cudaStreamDestroy(sA);
    cudaStreamDestroy(sB1);
    cudaStreamDestroy(sB2);
    cudaStreamDestroy(sC);
    cudaFree(d_produced);
    cudaFree(d_consumed);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o inter_stream_sync inter_stream_sync.cu && ./inter_stream_sync

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python/Numba Event Timing Demo (OPTIONAL)

@cuda.jit
def heavy_kernel(data):
    tid = cuda.grid(1)
    if tid < data.shape[0]:
        val = data[tid]
        for _ in range(1000):
            val = val * 1.0001 + 0.0001
        data[tid] = val

n = 1 << 20

# Create device array
d_data = cuda.device_array(n, dtype=np.float32)

# Warmup
heavy_kernel[(n+255)//256, 256](d_data)
cuda.synchronize()

# Time with events
stream = cuda.stream()
start_event = stream.record()

for _ in range(10):
    heavy_kernel[(n+255)//256, 256, stream](d_data)

end_event = stream.record()
stream.synchronize()

elapsed = cuda.event_elapsed_time(start_event, end_event)
print(f"10 kernels: {elapsed:.2f} ms")
print(f"Per kernel: {elapsed/10:.2f} ms")

---

## Part 4: Event Flags & Advanced Usage

### Event Flags

```cpp
// event_flags.cu - Different event types

// cudaEventDefault - Standard event for timing and sync
cudaEvent_t defaultEvent;
cudaEventCreate(&defaultEvent);

// cudaEventBlockingSync - CPU thread yields instead of spinning
// Better for power efficiency, slightly higher latency
cudaEvent_t blockingEvent;
cudaEventCreateWithFlags(&blockingEvent, cudaEventBlockingSync);

// cudaEventDisableTiming - Can't use for timing, but faster
// Good for pure synchronization
cudaEvent_t syncOnlyEvent;
cudaEventCreateWithFlags(&syncOnlyEvent, cudaEventDisableTiming);

// Combine flags
cudaEvent_t fastSyncEvent;
cudaEventCreateWithFlags(&fastSyncEvent, 
                          cudaEventBlockingSync | cudaEventDisableTiming);
```

### When to Use Each

```
┌─────────────────────────────────────────────────────────┐
│                    EVENT FLAGS                          │
├──────────────────────┬──────────────────────────────────┤
│ Flag                 │ Use Case                         │
├──────────────────────┼──────────────────────────────────┤
│ Default              │ Timing + sync, general use       │
│ BlockingSync         │ Long waits, power-sensitive      │
│ DisableTiming        │ Pure sync, maximum performance   │
│ Both combined        │ Sync-only, power-efficient       │
└──────────────────────┴──────────────────────────────────┘
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile events_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>
#include <float.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
            exit(1); \
        } \
    } while(0)

__global__ void computeKernel(float* data, int n, int iters) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = data[idx];
        for (int i = 0; i < iters; i++) {
            val = sinf(val) * cosf(val) + 0.1f;
        }
        data[idx] = val;
    }
}

// ============================================================
// Exercise 1: Benchmark Kernel Utility
// ============================================================

struct BenchmarkResult {
    float minMs;
    float maxMs;
    float avgMs;
    float stddevMs;
};

BenchmarkResult benchmarkKernel(void (*launchFunc)(float*, int), 
                                  float* d_data, int n,
                                  int warmupIters, int benchIters) {
    // Warmup
    for (int i = 0; i < warmupIters; i++) {
        launchFunc(d_data, n);
    }
    cudaDeviceSynchronize();
    
    float* times = (float*)malloc(benchIters * sizeof(float));
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    for (int i = 0; i < benchIters; i++) {
        cudaEventRecord(start);
        launchFunc(d_data, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&times[i], start, stop);
    }
    
    // Calculate statistics
    BenchmarkResult result;
    result.minMs = FLT_MAX;
    result.maxMs = 0;
    result.avgMs = 0;
    
    for (int i = 0; i < benchIters; i++) {
        if (times[i] < result.minMs) result.minMs = times[i];
        if (times[i] > result.maxMs) result.maxMs = times[i];
        result.avgMs += times[i];
    }
    result.avgMs /= benchIters;
    
    // Standard deviation
    float variance = 0;
    for (int i = 0; i < benchIters; i++) {
        float diff = times[i] - result.avgMs;
        variance += diff * diff;
    }
    result.stddevMs = sqrtf(variance / benchIters);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    free(times);
    
    return result;
}

void launchCompute(float* data, int n) {
    computeKernel<<<(n + 255) / 256, 256>>>(data, n, 100);
}

void exercise1_benchmarkUtility() {
    printf("=== Exercise 1: Benchmark Kernel Utility ===\n");
    
    const int n = 1 << 20;
    float* d_data;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    
    BenchmarkResult result = benchmarkKernel(launchCompute, d_data, n, 5, 20);
    
    printf("Kernel Benchmark Results (20 iterations, 5 warmup):\n");
    printf("  Min:     %.3f ms\n", result.minMs);
    printf("  Max:     %.3f ms\n", result.maxMs);
    printf("  Avg:     %.3f ms\n", result.avgMs);
    printf("  Stddev:  %.3f ms\n", result.stddevMs);
    printf("  Range:   %.3f ms (%.1f%% of avg)\n\n", 
           result.maxMs - result.minMs,
           (result.maxMs - result.minMs) / result.avgMs * 100);
    
    cudaFree(d_data);
}

// ============================================================
// Exercise 2: DAG Executor
// ============================================================

__global__ void kernelA(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] + 1.0f;
}

__global__ void kernelB(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = data[idx] * 2.0f;
}

__global__ void kernelC(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = sinf(data[idx]);
}

__global__ void kernelD(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) data[idx] = sqrtf(fabsf(data[idx]));
}

__global__ void kernelE(const float* b, const float* c, const float* d, 
                         float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) result[idx] = b[idx] + c[idx] + d[idx];
}

void exercise2_dagExecutor() {
    printf("=== Exercise 2: DAG Executor ===\n");
    printf("DAG Structure:\n");
    printf("     A\n");
    printf("    /|\\\n");
    printf("   B C D\n");
    printf("    \\|/\n");
    printf("     E\n\n");
    
    const int n = 1 << 20;
    int grid = (n + 255) / 256;
    
    float *d_data, *d_b, *d_c, *d_d, *d_result;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_c, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_d, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_result, n * sizeof(float)));
    
    // Create streams for parallel branches
    cudaStream_t streamB, streamC, streamD, streamE;
    cudaStreamCreate(&streamB);
    cudaStreamCreate(&streamC);
    cudaStreamCreate(&streamD);
    cudaStreamCreate(&streamE);
    
    // Events for synchronization
    cudaEvent_t eventA, eventB, eventC, eventD;
    cudaEventCreate(&eventA);
    cudaEventCreate(&eventB);
    cudaEventCreate(&eventC);
    cudaEventCreate(&eventD);
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    
    // A executes first (on default stream)
    kernelA<<<grid, 256>>>(d_data, n);
    cudaEventRecord(eventA);
    
    // B, C, D wait for A and run in parallel
    cudaStreamWaitEvent(streamB, eventA);
    cudaMemcpyAsync(d_b, d_data, n * sizeof(float), cudaMemcpyDeviceToDevice, streamB);
    kernelB<<<grid, 256, 0, streamB>>>(d_b, n);
    cudaEventRecord(eventB, streamB);
    
    cudaStreamWaitEvent(streamC, eventA);
    cudaMemcpyAsync(d_c, d_data, n * sizeof(float), cudaMemcpyDeviceToDevice, streamC);
    kernelC<<<grid, 256, 0, streamC>>>(d_c, n);
    cudaEventRecord(eventC, streamC);
    
    cudaStreamWaitEvent(streamD, eventA);
    cudaMemcpyAsync(d_d, d_data, n * sizeof(float), cudaMemcpyDeviceToDevice, streamD);
    kernelD<<<grid, 256, 0, streamD>>>(d_d, n);
    cudaEventRecord(eventD, streamD);
    
    // E waits for B, C, D
    cudaStreamWaitEvent(streamE, eventB);
    cudaStreamWaitEvent(streamE, eventC);
    cudaStreamWaitEvent(streamE, eventD);
    kernelE<<<grid, 256, 0, streamE>>>(d_b, d_c, d_d, d_result, n);
    
    cudaEventRecord(stop, streamE);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    printf("DAG execution time: %.2f ms\n\n", ms);
    
    cudaStreamDestroy(streamB);
    cudaStreamDestroy(streamC);
    cudaStreamDestroy(streamD);
    cudaStreamDestroy(streamE);
    cudaEventDestroy(eventA);
    cudaEventDestroy(eventB);
    cudaEventDestroy(eventC);
    cudaEventDestroy(eventD);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_data);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_d);
    cudaFree(d_result);
}

// ============================================================
// Exercise 3: Event Pool
// ============================================================

class EventPool {
private:
    cudaEvent_t* events;
    bool* inUse;
    int poolSize;
    int nextFree;
    
public:
    EventPool(int size) : poolSize(size), nextFree(0) {
        events = new cudaEvent_t[size];
        inUse = new bool[size];
        for (int i = 0; i < size; i++) {
            cudaEventCreate(&events[i]);
            inUse[i] = false;
        }
    }
    
    ~EventPool() {
        for (int i = 0; i < poolSize; i++) {
            cudaEventDestroy(events[i]);
        }
        delete[] events;
        delete[] inUse;
    }
    
    cudaEvent_t acquire() {
        // Find a free event
        for (int i = 0; i < poolSize; i++) {
            int idx = (nextFree + i) % poolSize;
            if (!inUse[idx]) {
                inUse[idx] = true;
                nextFree = (idx + 1) % poolSize;
                return events[idx];
            }
        }
        // Pool exhausted - create a new event (shouldn't happen with proper sizing)
        printf("Warning: Event pool exhausted!\n");
        cudaEvent_t newEvent;
        cudaEventCreate(&newEvent);
        return newEvent;
    }
    
    void release(cudaEvent_t event) {
        for (int i = 0; i < poolSize; i++) {
            if (events[i] == event) {
                inUse[i] = false;
                return;
            }
        }
    }
};

void exercise3_eventPool() {
    printf("=== Exercise 3: Event Pool ===\n");
    
    const int poolSize = 8;
    const int numOperations = 100;
    
    EventPool pool(poolSize);
    
    const int n = 1 << 18;
    float* d_data;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    
    cudaEvent_t overallStart, overallStop;
    cudaEventCreate(&overallStart);
    cudaEventCreate(&overallStop);
    
    float totalTime = 0;
    
    cudaEventRecord(overallStart);
    
    for (int i = 0; i < numOperations; i++) {
        cudaEvent_t start = pool.acquire();
        cudaEvent_t stop = pool.acquire();
        
        cudaEventRecord(start);
        computeKernel<<<(n + 255) / 256, 256>>>(d_data, n, 10);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        
        float ms;
        cudaEventElapsedTime(&ms, start, stop);
        totalTime += ms;
        
        pool.release(start);
        pool.release(stop);
    }
    
    cudaEventRecord(overallStop);
    cudaEventSynchronize(overallStop);
    
    float overallMs;
    cudaEventElapsedTime(&overallMs, overallStart, overallStop);
    
    printf("Event Pool (size=%d):\n", poolSize);
    printf("  Operations: %d\n", numOperations);
    printf("  Kernel time (sum): %.2f ms\n", totalTime);
    printf("  Overall time:      %.2f ms\n", overallMs);
    printf("  Events reused: %d times per event\n\n", numOperations * 2 / poolSize);
    
    cudaEventDestroy(overallStart);
    cudaEventDestroy(overallStop);
    cudaFree(d_data);
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║              CUDA Events Exercises                           ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n\n", prop.name);
    
    exercise1_benchmarkUtility();
    exercise2_dagExecutor();
    exercise3_eventPool();
    
    printf("══════════════════════════════════════════════════════════════\n");
    printf("                    All exercises completed!\n");
    printf("══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o events_exercises events_exercises.cu && ./events_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Benchmark Kernel
Create a timing utility that:
- Warms up the kernel
- Runs multiple iterations
- Reports min/max/avg times

### Exercise 2: DAG Executor
```cpp
// Implement a simple DAG (Directed Acyclic Graph) of kernels:
//     A
//    /|\
//   B C D
//    \|/
//     E
```

### Exercise 3: Event Pool
Implement an event pool that reuses events to avoid creation overhead.

---

## 🎯 Key Takeaways

```
╔══════════════════════════════════════════════════════════════════╗
║                   CUDA EVENTS MASTERY                            ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ⏱️  Core Concept: STOPWATCHES + CHECKPOINTS                      ║
║     Events time kernels AND synchronize dependent streams        ║
║                                                                  ║
║  📋 Essential API:                                               ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  // Lifecycle                                              │  ║
║  │  cudaEventCreate(&event)                                   │  ║
║  │  cudaEventRecord(event, stream)     // Mark a point        │  ║
║  │  cudaEventDestroy(event)                                   │  ║
║  │                                                            │  ║
║  │  // Synchronization                                        │  ║
║  │  cudaEventSynchronize(event)        // CPU waits           │  ║
║  │  cudaStreamWaitEvent(stream, event) // Stream waits        │  ║
║  │                                                            │  ║
║  │  // Timing                                                 │  ║
║  │  cudaEventElapsedTime(&ms, start, stop)  // Microseconds!  │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  🔀 Inter-Stream Dependencies (DAG Pattern):                     ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  Stream A: [Work]─🏁─────────────────────────────          │  ║
║  │  Stream B: [Work]─🏁────────────────────────               │  ║
║  │  Stream C:        wait(A)─wait(B)─[Depends on A & B]       │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  ⚡ Timing Best Practices:                                       ║
║     • Always warmup before timing                                ║
║     • Run multiple iterations                                    ║
║     • Use events NOT CPU timers for GPU work                     ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## 🔮 What's Next?

**Day 5: CUDA Graphs** - Compile your workflow!

You've learned to orchestrate streams with events. But each kernel launch has ~5-10μs CPU overhead. For workloads with MANY small kernels, this adds up!

```
Today (Events):                 Tomorrow (CUDA Graphs):
Launch kernel 1: 5μs overhead   Capture entire workflow once:
Launch kernel 2: 5μs overhead   ┌──────────────────────────┐
Launch kernel 3: 5μs overhead   │  K1 → K2 → K3 → K4 → K5  │
Launch kernel 4: 5μs overhead   └──────────────────────────┘
Launch kernel 5: 5μs overhead   
─────────────────────────────   Launch graph: 10μs TOTAL!
Total overhead: 25μs            Replay graph: 10μs each time!
```

Turn your stream workflow into a "recipe card" that executes with minimal overhead! 📋